In [1]:
import pandas as pd
import stanza
import re
import os
import glob
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

In [2]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

2021-04-15 23:56:02 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-04-15 23:56:02 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2021-04-15 23:56:02 INFO: Use device: cpu
2021-04-15 23:56:02 INFO: Loading: tokenize
2021-04-15 23:56:02 INFO: Loading: pos
2021-04-15 23:56:02 INFO: Loading: lemma
2021-04-15 23:56:02 INFO: Done loading processors!


In [3]:
gt = pd.read_csv('../ACTER/en/htfl/annotations/htfl_en_terms.ann', sep='	', engine='python',header=None)

In [4]:
preds = pd.read_csv('../results/en/ann/5_lem_roberta_ann_predictions.csv')['predictions']
preds.head(5)

0               blocker
1                anemia
2    arterial tonometry
3        glucocorticoid
4              nitroxyl
Name: predictions, dtype: object

In [5]:
path = "../ACTER/en/htfl/texts/annotated/"
list_of_files = os.listdir("../ACTER/en/htfl/texts/annotated/")
lines=[]
for file in list_of_files:
    f = open(path+file, "r")
    #append each line in the file to a list
    lines.append(f.readlines())
    f.close()

In [8]:
def nextword(target, source):
    for i, w in enumerate(source):
        if w == target:
            return source[i+1]
        
def beforeword(target, source):
    for i, w in enumerate(source):
        if w == target:
            if i>0:
                return source[i-1]
            
preds = preds.to_list()
for line in lines:
    for pred in preds:
        n_word = str(nextword(pred,line))
        b_word = str(beforeword(pred,line))
        if nlp(n_word).sentences[0].words[0].text != 'None': 
            if nlp(n_word).sentences[0].words[0].upos == 'NOUN':
                print('1')
                preds.append(pred + ' '+ n_word)
        if nlp(n_word).sentences[0].words[0].text != 'None': 
            if nlp(b_word).sentences[0].words[0].upos == 'NOUN'or nlp(b_word).sentences[0].words[0].upos == 'ADJ':
                print('2')
                preds.append(b_word + ' ' + pred)
len(preds)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/hanhtran/miniconda3/envs/term/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-72ae1e4db857>", line 9, in <module>
    if nlp(n_word).sentences[0].words[0].text != 'None':
  File "/Users/hanhtran/miniconda3/envs/term/lib/python3.9/site-packages/stanza/pipeline/core.py", line 210, in __call__
    doc = self.process(doc)
  File "/Users/hanhtran/miniconda3/envs/term/lib/python3.9/site-packages/stanza/pipeline/core.py", line 204, in process
    doc = process(doc)
  File "/Users/hanhtran/miniconda3/envs/term/lib/python3.9/site-packages/stanza/pipeline/pos_processor.py", line 33, in process
    preds += self.trainer.predict(b)
  File "/Users/hanhtran/miniconda3/envs/term/lib/python3.9/site-packages/stanza/models/pos/trainer.py", line 73, in predict
    _, preds = self.model(word, word_mask, wordchars, wordchars_mask, upos, xpos,

TypeError: object of type 'NoneType' has no len()

In [ ]:
nlp(b_word).sentences[0].words[0].text

Tu BIO tim nhung tu dang truoc